In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import random
import sympy
import scipy.stats as st
import scipy.special
import pandas
import csv
import math
import multiprocessing as mp
import datetime
from decimal import *
from mpl_toolkits.mplot3d import Axes3D
from google.colab import drive

## **Google Drive**

In [2]:
# connect to google drive to save the csv file
drive.mount('drive')
os.chdir("drive/My Drive/Yen-Ting Lin/Code")

Mounted at drive


## **Initial Parameters**

In [25]:
# number of preamble
N = 54

# number of max retransmission
retrans = 3 # N_PTmax - 1

# Time Duration (ms)
T_RAREP = 1280
T_RAR = 6.4 + 3
# pp = Rmax * G = 1 * 8
# TS 36.331, ra-ResponseWindowSize-r13 = 2pp
W_RAR = 16
# TS 36.331, max-ContentionResoulutionTimer-r13 = 4pp
T_N_CR = 32
# TS 36.321, Backoff Parameter Values for NB-IoT
W_BO = 4096

def maxslot():
  Maxraslot = np.ceil((T_RAR + W_RAR + W_BO) / T_RAREP) * (retrans + 1) + 1
  return int(Maxraslot)

# initail number of UE
M = []
for i in range(1, 1001):
  M.append(i)
# print(len(M))

## **Back-off Calculation (Type 1)**

In [4]:
# alpha without i

def find_kmin_type1(i):
  return np.floor((i + 1) + ((T_RAR + W_RAR - 1) / T_RAREP))

def find_kmax_type1(i):
  return np.ceil(i + ((T_RAR + W_RAR + W_BO) / T_RAREP))

def find_alpha_type1(k, kmin, kmax):
  if k == kmin:
    return (T_RAREP - ((T_RAR + W_RAR) % T_RAREP)) / W_BO
  elif k == kmax:
    return ((T_RAR + W_RAR + W_BO) % T_RAREP) / W_BO
  elif k < kmax and k > kmin:
    return (T_RAREP) / W_BO
  else:
    return 0

## **Main Function (For type 1)**

In [26]:
def main_type1():
  Ms_list = [0.0 for i in range(0, len(M))]
  Ps_list = [0.0 for i in range(0, len(M))]
  Da_list = [0.0 for i in range(0, len(M))]

  for ue in range(0, len(M)):
    # initial the MiSn and MiFn: Mi[slot][num of trans + 1]
    # the last time of retrans needed to be ignored
    MiCn = [[0.0] * int((retrans + 1) + 1) for _ in range(0, maxslot() + 1)]
    MiSn = [[0.0] * int((retrans + 1) + 1) for _ in range(0, maxslot() + 1)]
    MiFn = [[0.0] * int((retrans + 1) + 1) for _ in range(0, maxslot() + 1)]
    T = [0.0 for i in range(0, maxslot() + 1)]

    # num of total UE in slot 0
    MiFn[0][0] = M[ue]
    # print(np.matrix(MiFn))

    for i in range(0, maxslot()):
      # print("i = ", i, "\n")
      kmin = find_kmin_type1(i)
      kmax = find_kmax_type1(i)
      # print("kmin = ", kmin, "kmax = ", kmax, "\n")
      m = MiFn[i][0] + MiFn[i][1] + MiFn[i][2] + MiFn[i][3] # Mi
      for n in range(0, int(retrans + 1)):
        # print("n = ", n, "\n")
        # RA procedure
        if MiFn[i][n] != 0:
          for k in range(int(kmin), int(kmax + 1)):
            # print("k = ", k, "\n")
            if N == 0:
              E_M = 0
            else:
              E_M = np.exp(-(m / N))
            alpha = find_alpha_type1(k, kmin, kmax)
            MiFn[k][n + 1] += alpha * (MiFn[i][n] * (1.0 - E_M))
            # print("alpha = ", alpha, "\n")
            # print("MiCn = ", MiFn[i][n] * (1.0 - E_M), "\n")
            # print(np.matrix(MiFn))
      for n in range(0, int(retrans + 1)):
        if N == 0:
          E_M = 0
        else:
          E_M = np.exp(-(m / N))
        MiSn[i][n] = MiFn[i][n] * E_M
        MiCn[i][n] = MiFn[i][n] * (1.0 - E_M)
      # define delay
      T[i] = (i + 1) * T_RAREP + T_RAR + W_RAR

    # Ms, Ps, Da
    for i in range(0, maxslot()):
      for n in range(0, int(retrans + 1)):
        # sum the number of succeed UE for every transmission in each slot
        Ms_list[ue] += MiSn[i][n]
        Da_list[ue] += MiSn[i][n] * float(T[i])
    # total number of succeed UE when there have k UE in the system
    Ms_list[ue] = round(Ms_list[ue] * 1000000) / 1000000
    if M[ue] != 0:
      Ps_list[ue] = round((Ms_list[ue] / M[ue]) * 1000000) / 1000000
    else:
      Ps_list[ue] = 0
    if M[ue] != 0:
      Da_list[ue] = float(Da_list[ue]) / float(Ms_list[ue])
    else:
      Da_list[ue] = 0.0

  return M, Ms_list, Ps_list, Da_list

## **Test Result (For type 1)**

In [27]:
columns = ["M", "Ms", "Ps", "Da"]
# M, Ms_list, Ps_list, Da_list = main_type1()
M, Ms_list, Ps_list, Da_list = main_type1()

data = {"M" : M, "Ms" : Ms_list, "Ps" : Ps_list, "Da" : Da_list}
result = pandas.DataFrame(data)
# cvs_pandas = result.to_csv("ana_testresult.csv", mode='a', index=False, header=True, columns=columns)
cvs_pandas = result.to_csv("ana_testresult.csv", mode='w+', index=False, header=True, columns=columns)

## **Plotting Figures**

In [28]:
ana = pandas.read_csv('ana_testresult.csv')
# sim = pandas.read_csv('sim_testresult.csv')
ana

,M,Ms,Ps,Da
0,1,1.000000,1.000000,1355.748785
1,2,2.000000,1.000000,1405.203285
2,3,3.000000,1.000000,1453.807927
3,4,4.000000,1.000000,1501.604807
4,5,5.000000,1.000000,1548.633821
...,...,...,...,...
995,996,51.373745,0.051580,12788.366728
996,997,51.343734,0.051498,12791.313858
997,998,51.313809,0.051417,12794.252754
998,999,51.283968,0.051335,12797.183850


## **Back-off Calculation (Type 2)**

In [29]:
# alpha with i

def find_kmin_type2(i):
  return np.ceil((i - 1) + ((1 - (T_RAR + W_RAR + W_BO)) / T_RAREP))

def find_kmax_type2(i):
  return np.floor(i - ((T_RAR + W_RAR + 1) / T_RAREP))

def find_alpha_type2(i, k, kmin, kmax):
  if kmin <= k <= (i - ((T_RAR + W_RAR + W_BO) / T_RAREP)):
    return (((k - 1) * T_RAREP) + T_RAR + W_RAR + W_BO - ((i - 2) * T_RAREP)) / W_BO
  elif (i - ((T_RAR + W_RAR + W_BO) / T_RAREP)) < k < ((i - 1) - ((T_RAR + W_RAR) / T_RAREP)):
    return ((T_RAREP) / W_BO)
  elif ((i - 1) - ((T_RAR + W_RAR) / T_RAREP)) <= k <= kmax:
    return (((i - 1) * T_RAREP) - (((k - 1) * T_RAREP) + T_RAR + W_RAR)) / W_BO
  else:
    return 0

## **Main Function (For type 2)**

In [30]:
def main_type2():
  Ms_list = [0.0 for i in range(0, len(M))]
  Ps_list = [0.0 for i in range(0, len(M))]
  Da_list = [0.0 for i in range(0, len(M))]

  for ue in range(0, len(M)):
    # initial the MiSn and MiFn: Mi[slot][num of trans + 1]
    # the last time of retrans needed to be ignored
    MiCn = [[0.0] * int((retrans + 1) + 1) for _ in range(0, maxslot() + 1)]
    MiSn = [[0.0] * int((retrans + 1) + 1) for _ in range(0, maxslot() + 1)]
    MiFn = [[0.0] * int((retrans + 1) + 1) for _ in range(0, maxslot() + 1)]
    T = [0.0 for i in range(0, maxslot() + 1)]

    # num of total UE in slot 0
    MiFn[0][0] = M[ue]
    # print(np.matrix(MiFn))

    for i in range(0, maxslot()):
      # print("i = ", i, "\n")
      kmin = find_kmin_type2(i)
      kmax = find_kmax_type2(i)
      # print("kmin = ", kmin, "kmax = ", kmax, "\n")
      m = MiFn[i][0] + MiFn[i][1] + MiFn[i][2] + MiFn[i][3] # Mi
      for n in range(0, int(retrans + 1)):
        # print("n = ", n, "\n")
        # RA procedure
        k_list = []
        if MiFn[i][n] != 0:
          for k in range(int(kmin), int(kmax + 1)):
            # print("k = ", k, "\n")
            if N == 0:
              E_M = 0
            else:
              E_M = np.exp(-(m / N))
            alpha = find_alpha_type2(i, k, kmin, kmax)
            k_list.append(alpha * (MiFn[i][n] * (1.0 - E_M)))
            # print("alpha = ", alpha, "\n")
          # process the data to put it into array like type 1
          for itr_k in range(0, len(k_list)):
            MiFn[i + itr_k + 1][n + 1] += k_list[(len(k_list) - 1) - itr_k]
            # print("MiCn = ", MiFn[i][n] * (1.0 - E_M), "\n")
            # print(np.matrix(MiFn))
      for n in range(0, int(retrans + 1)):
        if N == 0:
          E_M = 0
        else:
          E_M = np.exp(-(m / N))
        MiSn[i][n] = MiFn[i][n] * E_M
        MiCn[i][n] = MiFn[i][n] * (1.0 - E_M)
      # define delay
      T[i] = (i + 1) * T_RAREP + T_RAR + W_RAR

    # Ms, Ps, Da
    for i in range(0, maxslot()):
      for n in range(0, int(retrans + 1)):
        # sum the number of succeed UE for every transmission in each slot
        Ms_list[ue] += MiSn[i][n]
        Da_list[ue] += MiSn[i][n] * float(T[i])
    # total number of succeed UE when there have k UE in the system
    Ms_list[ue] = round(Ms_list[ue] * 1000000) / 1000000
    if M[ue] != 0:
      Ps_list[ue] = round((Ms_list[ue] / M[ue]) * 1000000) / 1000000
    else:
      Ps_list[ue] = 0
    if M[ue] != 0:
      Da_list[ue] = float(Da_list[ue]) / float(Ms_list[ue])
    else:
      Da_list[ue] = 0.0

    # print(len(M), len(Ms_list), len(Ps_list), len(Da_list))

  return M, Ms_list, Ps_list, Da_list

## **Test Result (For type 2)**

In [31]:
columns = ["M", "Ms", "Ps", "Da"]
M, Ms_list, Ps_list, Da_list = main_type2()

data = {"M" : M, "Ms" : Ms_list, "Ps" : Ps_list, "Da" : Da_list}
result = pandas.DataFrame(data)
# cvs_pandas = result.to_csv("ana_testresult.csv", mode='a', index=False, header=True, columns=columns)
cvs_pandas = result.to_csv("ana_testresult.csv", mode='w+', index=False, header=True, columns=columns)

## **Plotting Figures**

In [32]:
ana = pandas.read_csv('ana_testresult.csv')
# sim = pandas.read_csv('sim_testresult.csv')
ana

,M,Ms,Ps,Da
0,1,1.000000,1.000000,1355.748785
1,2,2.000000,1.000000,1405.203285
2,3,3.000000,1.000000,1453.807927
3,4,4.000000,1.000000,1501.604807
4,5,5.000000,1.000000,1548.633821
...,...,...,...,...
995,996,51.373745,0.051580,12788.366728
996,997,51.343734,0.051498,12791.313858
997,998,51.313809,0.051417,12794.252754
998,999,51.283968,0.051335,12797.183850
